**ESTE NOTEBOOK NO ES NECESARIO EJECUTARLO CON LA GPU**

# Etiquetado de las pymes españolas

Este notebook tiene como objetivo, aplicar el criterio principal del estudio mencionado en la sección **4.1.4** sobre las variables de la sección **6.2.1**, para identificar y etiquetar a las pymes zombis.

Además, también se identifican zombis con el segundo criterio propuesto, con el fin de comparar el número de pymes zombis identificadas por ambos criterios, y asi llegar a la conclusión de si dichos criterios pueden funcionar de manera alternativa. 

Pero antes de continuar, es necesario recalcar nuevamente, que dichas variables se emplean con el fin único de etiquetar a las pymes zombis (un **1**) y no zombis (un **0**) del dataset total de **10907** pymes extraídas de SABI. Así, una vez que todas las pymes tengan una etiqueta asignada, se procederá con la extracción de las variables enumeradas en la sección **6.2.2**. Estas últimas son las que se emplearán finalmente como entrada a la red neuronal convolutiva, cuya principal intención será predecir con datos contables de los años anteriores (2017-2019) que empresas se convertirán en zombis en el año 2019.

## Importación de librerías

A continuación, se procede con la importación de las librerías necesarias para la ejecución correcta del notebook

In [1]:
import pandas as pd

In [2]:
!pip install openpyxl

     |████████████████████████████████| 242 kB 923 kB/s            


## Identificación y etiquetado de las pymes zombis españolas: Criterio principal

### Carga de los tipos de intereses

En primer lugar, es necesario cargar en un *dataframe* de pandas, los tipos de intereses medios anuales desde el año 2016 hasta el 2019. Así, tal y como se menciona en la sección **4.1.4**, dichos tipos de intereses consisten en datos externos a SABI y que han sido recolectados desde la página oficial de Banco de España.

Estos tipos de intereses se emplearán para el cálculo de la variable **R*** para cada empresa de la muestra y para los años: 2019, 2018 y 2017, respectivamente.

**SI SE EJECUTA ESTE NOTEBOOK EN LOCAL PUES SE DEBE CAMBIAR LA RUTA**

In [3]:
interest_rates = pd.read_excel('../input/datos-notebook-1/tipos_intereses.xlsx', "tipos_criterio_1")

Como se puede observar en la celda de abajo, el Banco de España ofrece los tipos de intereses aplicados por las instituciones financieras monetarias residentes en la Unión Económica y Monetaria (UEM) sobre los préstamos y créditos concedidos a las sociedades no financieras, en **dos grupos temporales** bien diferenciados:
- Créditos a corto plazo (c/p)
- Créditos a largo plazo (l/p)

Además, cada uno de los grupos temporales anteriores, se encuentran divididos en **tres posibles rangos de créditos**:
- Créditos <= 250.000 euros
- Créditos < 250.000 <= 1.000.000 euros
- Créditos > 1.000.000 euros

Por lo tanto, al calcular la variable **R*** para cada una de las empresas y año (**ecuación 8** de la memoria), habrá que tener en cuenta la temporalidad y el rango entre la que se encuentra su deuda remunerada, para posteriormente multiplicarlas por el tipo de interés que corresponda.

Un último detalle a destacar con respecto a los tipos de intereses cargados previamente es que, tal y como se puede apreciar en la **ecuación 8** de la memoria, el factor que multiplica a la deuda remunerada a largo plazo consiste en una media de los tipos de intereses a largo plazo de los cinco años anteriores. En este sentido, y para una mayor facilidad del notebook, dichas medias ya han sido previamente calculadas en Excel y son las que figuran en los intereses a largo plazo del ***dataframe: interest_rates***.   


In [4]:
interest_rates.head()

,Año,Créditos c/p ≤ 250.000 euros,250.000 < Créditos c/p ≤ 1M euros,Créditos c/p > 1M euros,Créditos l/p ≤ 250.000 euros,250.000 < Créditos l/p ≤ 1M euros,Créditos l/p > 1M euros
0,2016,2.92000,1.885917,1.684083,5.729200,4.435642,3.244300
1,2017,2.43025,1.671833,1.633833,5.031867,3.744883,2.862692
2,2018,2.11250,1.551167,1.585583,4.324275,3.071658,2.452083
3,2019,1.98075,1.495833,1.469667,3.613233,2.377417,2.020025


### Carga de las pymes iniciales descargadas de SABI

El siguiente paso es cargar en otro *dataframe* de pandas las variables que han sido extraídas en la sección **6.2.1**. Asi, estas variables son las que permitirán al estudio diferenciar entre las pymes zombis y no zombis.

Como se puede observar, el *dataframe*: **sme**, posee todas aquellas variables mencionadas en la sección **6.2.1** además del **Código NIF** y **Nombre** de la empresa.

**SI SE EJECUTA ESTE NOTEBOOK EN LOCAL PUES SE DEBE CAMBIAR LA RUTA**

In [5]:
sme = pd.read_excel('../input/datos-notebook-1/pymes_iniciales.xlsx', 'variables')
sme.head()

,Código NIF,Nombre,ebit 2019,ebit 2018,ebit 2017,deuda_c/p 2019,deuda_c/p 2018,deuda_c/p 2017,deuda_c/p 2016,deuda_con_grupo_c/p 2019,...,deuda_con_grupo_c/p 2016,deuda_l/p 2019,deuda_l/p 2018,deuda_l/p 2017,deuda_l/p 2016,deuda_con_grupo_l/p 2019,deuda_con_grupo_l/p 2018,deuda_con_grupo_l/p 2017,deuda_con_grupo_l/p 2016,gastos_financieros 2019
0,A01007137,"HOTELES VASCO-CATALANES, SA",859610.58,2038175.84,2193826.07,221309.04,239764.46,243580.33,226284.97,n.d.,...,n.d.,n.d.,n.d.,n.d.,n.d.,n.d.,n.d.,n.d.,n.d.,n.d.
1,A01012764,"HOTEL HELIOS C'AN PASTILLA, SA",1317266.62,2094048.12,1962945.93,215630.61,201823.32,211778.59,120633.96,n.d.,...,n.d.,2271.83,2271.83,2271.83,2271.83,n.d.,n.d.,n.d.,n.d.,n.d.
2,A01045772,BAR RESTAURANTE LOS GUARANIS SA,-5106.77,-36794.05,-14717.25,8500,2123.97,1146.18,8938.08,n.d.,...,n.d.,8376.42,25335.05,33564.5,33670.97,n.d.,n.d.,n.d.,n.d.,-2272.58
3,A01104157,EXPLOTACIONES SUBIJANA DE ALAVA SA,979628.32,1131808.94,841951.59,1259167.65,1802709.72,1615663.79,846415.71,n.d.,...,n.d.,13590231.27,8672240.51,9149877.46,10377394.89,n.d.,n.d.,n.d.,n.d.,-157270.5
4,A01280577,BIASTERI TURISMO SA,209594.38,121434.18,178080.51,333764.6,743774.29,714753.39,557864.83,n.d.,...,n.d.,2785539.59,2030134.58,2253090.37,2553240.82,n.d.,n.d.,n.d.,n.d.,-168325.77


### Limpieza de datos y cálculo de las variables necesarias

Se define la función ***calculateTotalDebt(debt1, debt2):*** que devuelve el importe total de la deuda ya sea a corto o largo  plazo.

Además, para la correcta identificación de las pymes zombis, todas aquellas empresas de la muestra que dispongan del valor: "no disponible (n.d.)" entre alguna de sus deudas se sustituye esta por un 0

In [6]:
def calculateTotalDebt(debt1, debt2):
    if debt1 == 'n.d.': debt1 = 0
    if debt2 == 'n.d.': debt2 = 0
    return debt1 + debt2

Se calculan las variables:
- Deuda remunerada total a corto plazo (drt_cp)
- Deuda remunerada total a largo plazo (drt_lp)
- Deuda remunerada total (drt): solo para los años 2018 y 2019

In [7]:
sme['drt_cp 2019'] = sme[['deuda_c/p 2019','deuda_con_grupo_c/p 2019']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_cp 2018'] = sme[['deuda_c/p 2018','deuda_con_grupo_c/p 2018']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_cp 2017'] = sme[['deuda_c/p 2017','deuda_con_grupo_c/p 2017']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_cp 2016'] = sme[['deuda_c/p 2016','deuda_con_grupo_c/p 2016']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)

In [8]:
sme['drt_lp 2019'] = sme[['deuda_l/p 2019','deuda_con_grupo_l/p 2019']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_lp 2018'] = sme[['deuda_l/p 2018','deuda_con_grupo_l/p 2018']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_lp 2017'] = sme[['deuda_l/p 2017','deuda_con_grupo_l/p 2017']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt_lp 2016'] = sme[['deuda_l/p 2016','deuda_con_grupo_l/p 2016']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)

In [9]:
sme['drt 2019'] = sme[['drt_cp 2019','drt_lp 2019']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)
sme['drt 2018'] = sme[['drt_cp 2018','drt_lp 2018']].apply(lambda row: calculateTotalDebt(row[0],row[1]), axis=1)

En este punto, se eliminan todas las muestras que presentan el valor **n.d.** en los **gastos financieros**. Esto se debe a que dicha variable se emplea en el cálculo de **R** la cual a su vez se emplea en los criterios para identificar a las zombis. Por lo tanto, sustituir el valor faltante por un 0 no sería del todo adecuado y podríamos caer en el error de identificar a empresas no zombis como zombis o viceversa.

Esto mismo se realiza con la variable **EBIT 2019, 2018 y 2017**

In [10]:
print("Porcentaje de pymes con gastos financieros a n.d.:", (sme.loc[sme['gastos_financieros 2019'] == 'n.d.'].shape[0]/sme.shape[0])*100, "%")

Porcentaje de pymes con gastos financieros a n.d.: 15.53131016778216 %


In [11]:
sme.drop(sme.loc[sme['gastos_financieros 2019'] == 'n.d.'].index, inplace=True)
sme.shape

(9213, 32)

In [12]:
sme.drop(sme.loc[sme['ebit 2019'] == 'n.d.'].index, inplace=True)
sme.drop(sme.loc[sme['ebit 2018'] == 'n.d.'].index, inplace=True)
sme.drop(sme.loc[sme['ebit 2017'] == 'n.d.'].index, inplace=True)
sme = sme.reset_index(drop=True)

Esto nos deja con un total de **9204** de muestras restantes

In [13]:
sme.shape

(9204, 32)

Se establecen los umbrales necesarios para poder diferenciar entre los tres rangos de créditos mencionados anteriormente:

In [14]:
credit_threshold_1 = 250000
credit_threshold_2 = 1000000

Además, se definen las siguientes funciones que se utilizarán para el cálculo de R* y R en ambos criterios propuestos en el estudio:

- ***calculateMinInterestPayment(drlp, drcp, year):*** devuelve el pago mínimo de intereses para una empresa, teniendo en cuenta su deuda remunerada a largo y corto plazo, y el año (**ecuación 8**).
- ***getLoanInterest(credit, is_short_term, year):*** devuelve el tipo de interés medio a aplicar, teniendo en cuenta el rango entre el que se encuentra el crédito, su duración (corto o largo plazo) y el año.
- ***calculateActualInterest(drt_2018,gf_2019):*** devuelve la variable R (**ecuación 7**) 

In [15]:
def getLoanInterest(credit, is_short_term, year):
    interest = None
    if is_short_term:
        if credit <= credit_threshold_1:
            interest = interest_rates.loc[interest_rates.Año==year, 'Créditos c/p ≤ 250.000 euros'].values[0]
        elif credit > credit_threshold_1 and credit <= credit_threshold_2:
            interest = interest_rates.loc[interest_rates.Año==year, '250.000 < Créditos c/p ≤ 1M euros'].values[0]
        else:
            interest = interest_rates.loc[interest_rates.Año==year, 'Créditos c/p > 1M euros'].values[0] 
    else:
        if credit <= credit_threshold_1:
            interest = interest_rates.loc[interest_rates.Año==year, 'Créditos l/p ≤ 250.000 euros'].values[0]
        elif credit > credit_threshold_1 and credit <= credit_threshold_2:
            interest = interest_rates.loc[interest_rates.Año==year, '250.000 < Créditos l/p ≤ 1M euros'].values[0]
        else:
            interest = interest_rates.loc[interest_rates.Año==year, 'Créditos l/p > 1M euros'].values[0]
    return interest/100

def calculateMinInterestPayment(drlp, drcp, year):
    rl = getLoanInterest(drlp, False, year-1)
    rs = getLoanInterest(drcp, True, year-1)
    return drcp*rs + drlp*rl

def calculateActualInterest(drt_2018, gf_2019):
    if drt_2018 == 0: return 0
    else: return (-gf_2019)/drt_2018

Se calcula la columna R* para los años: 2019, 2018 y 2017, y la columna R para el año 2019

In [16]:
sme['r 2019'] = sme[['drt 2018', 'gastos_financieros 2019']].apply(lambda row: calculateActualInterest(row[0],row[1]), axis=1)

In [17]:
sme['r* 2019'] = sme[['drt_lp 2018', 'drt_cp 2018']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2019), axis=1)
sme['r* 2018'] = sme[['drt_lp 2017', 'drt_cp 2017']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2018), axis=1)
sme['r* 2017'] = sme[['drt_lp 2016', 'drt_cp 2016']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2017), axis=1)

In [18]:
sme.head()

,Código NIF,Nombre,ebit 2019,ebit 2018,ebit 2017,deuda_c/p 2019,deuda_c/p 2018,deuda_c/p 2017,deuda_c/p 2016,deuda_con_grupo_c/p 2019,...,drt_lp 2019,drt_lp 2018,drt_lp 2017,drt_lp 2016,drt 2019,drt 2018,r 2019,r* 2019,r* 2018,r* 2017
0,A01045772,BAR RESTAURANTE LOS GUARANIS SA,-5106.77,-36794.05,-14717.25,8500,2123.97,1146.18,8938.08,n.d.,...,8376.42,25335.05,33564.50,33670.97,16876.42,27459.02,0.082763,1140.426100,1716.775927,2190.069149
1,A01104157,EXPLOTACIONES SUBIJANA DE ALAVA SA,979628.32,1131808.94,841951.59,1259167.65,1802709.72,1615663.79,846415.71,n.d.,...,13590231.27,8672240.51,9149877.46,10377394.89,14849398.92,10474950.23,0.015014,241234.029041,288330.033113,352636.517360
2,A01280577,BIASTERI TURISMO SA,209594.38,121434.18,178080.51,333764.6,743774.29,714753.39,557864.83,n.d.,...,2785539.59,2030134.58,2253090.37,2553240.82,3119304.19,2773908.87,0.060682,61317.770542,76448.515690,93355.657730
3,A02003663,"HOTELES Y EDIFICIOS, SA",233388.3,172710.84,144877.19,n.d.,20972.88,100819.62,100027.24,n.d.,...,826344.39,843899.78,870238.02,977568.56,826344.39,864872.66,0.023516,26364.770007,35039.567386,46282.233776
4,A02021574,HOTELES DE LA MANCHA SA,-11767.78,-89182.46,-119563.75,498100.37,464668.44,447806.23,416079.41,n.d.,...,1461092.81,1666090.97,1698952.54,1680491.83,1959193.18,2130759.41,0.003723,48061.720945,56122.346605,62367.107380


Se define la función ***detectZombieFirms(ebit, min_interest_payment, r_2019, drt_2019, drt_2018):*** que devuelve un 1 si la empresa cumple con las condiciones para ser considerada zombi y un 0 cuando no las cumple.

In [19]:
def detectZombieFirms(ebit, min_interest_payment, r_2019, drt_2019, drt_2018):
    for i in range(0,3):
        if ebit[i] - min_interest_payment[i] > 0:
            return 0
    if drt_2019 > drt_2018: return 1
    elif min_interest_payment[0] != 0 and r_2019 < min_interest_payment[0]/drt_2018:
        return 1
    else: return 0

Se calcula la etiqueta para cada una de las pymes disponibles en el dataframe: **sme**

In [20]:
sme['zombi'] = sme.apply(lambda row: detectZombieFirms([row['ebit 2019'], row['ebit 2018'], row['ebit 2017']],
                                                          [row['r* 2019'], row['r* 2018'], row['r* 2017']], row['r 2019'],
                                                          row['drt 2019'], row['drt 2018']), axis=1)

In [21]:
sme.head(7)

,Código NIF,Nombre,ebit 2019,ebit 2018,ebit 2017,deuda_c/p 2019,deuda_c/p 2018,deuda_c/p 2017,deuda_c/p 2016,deuda_con_grupo_c/p 2019,...,drt_lp 2018,drt_lp 2017,drt_lp 2016,drt 2019,drt 2018,r 2019,r* 2019,r* 2018,r* 2017,zombi
0,A01045772,BAR RESTAURANTE LOS GUARANIS SA,-5106.77,-36794.05,-14717.25,8500,2123.97,1146.18,8938.08,n.d.,...,25335.05,33564.50,33670.97,16876.42,27459.02,0.082763,1140.426100,1716.775927,2190.069149,0
1,A01104157,EXPLOTACIONES SUBIJANA DE ALAVA SA,979628.32,1131808.94,841951.59,1259167.65,1802709.72,1615663.79,846415.71,n.d.,...,8672240.51,9149877.46,10377394.89,14849398.92,10474950.23,0.015014,241234.029041,288330.033113,352636.517360,0
2,A01280577,BIASTERI TURISMO SA,209594.38,121434.18,178080.51,333764.6,743774.29,714753.39,557864.83,n.d.,...,2030134.58,2253090.37,2553240.82,3119304.19,2773908.87,0.060682,61317.770542,76448.515690,93355.657730,0
3,A02003663,"HOTELES Y EDIFICIOS, SA",233388.3,172710.84,144877.19,n.d.,20972.88,100819.62,100027.24,n.d.,...,843899.78,870238.02,977568.56,826344.39,864872.66,0.023516,26364.770007,35039.567386,46282.233776,0
4,A02021574,HOTELES DE LA MANCHA SA,-11767.78,-89182.46,-119563.75,498100.37,464668.44,447806.23,416079.41,n.d.,...,1666090.97,1698952.54,1680491.83,1959193.18,2130759.41,0.003723,48061.720945,56122.346605,62367.107380,1
5,A02113066,HOTEL YESTE SA,1894.48,10578.8,13132.11,n.d.,n.d.,436,3375.81,6500,...,304396.72,319362.80,344178.96,279962.21,310896.72,0.000782,9487.339716,12128.326410,15554.919010,1
6,A02113397,TRIGA SA,539305.63,435480.71,170798.23,733424.46,668375.65,569347.49,590439.52,n.d.,...,0.00,0.00,0.00,733424.46,668375.65,0.008707,10367.620291,9518.541120,11135.197314,0


A continuación, se visualizan el número de pymes zombis identificadas por el criterio principal del estudio

In [22]:
print("Pymes Totales:", sme.shape[0])
print("Pymes Zombi:", len(sme.loc[sme['zombi'] == 1]))
print("Pymes No Zombi:", len(sme.loc[sme['zombi'] == 0]))

Pymes Totales: 9204
Pymes Zombi: 753
Pymes No Zombi: 8451


Finalmente, se guardan en dos Excel separados, los NIF's y nombres de las pymes zombis y no zombis que se utilizarán para la extracción de las variables de la sección **6.2.2**.

In [23]:
sme_zombis = sme.loc[sme['zombi'] == 1, ['Código NIF', 'Nombre']]
sme_no_zombis = sme.loc[sme['zombi'] == 0, ['Código NIF', 'Nombre']]
sme_zombis.to_excel('./pymes_zombis.xlsx', index=False)
sme_no_zombis.to_excel('./pymes_no_zombis.xlsx', index=False)

Se guardan los códigos NIF de las empresas zombis identificadas con el primer criterio para posteriormente comparar el grado de semejanza de ambos criterios.

In [24]:
zombies_nif_1 = list(sme_zombis['Código NIF'])
zombies_nif_1[0:5]

['A02021574', 'A02113066', 'A03358744', 'A03462272', 'A04102547']

## Identificación de las pymes zombis españolas: Criterio alternativo

Este **segundo criterio** difiere del principal del estudio, en que para el cálculo de R* no se considera una media de los intereses medios anuales a largo plazo de los últimos cinco años, sino que se toma directamente el interés medio anual a largo plazo del periodo inmediatamente anterior. Es por dicha razón, que se sobrescribe el dataframe: **interest_rates** con estos nuevos tipos de intereses proporcionados también por el Banco de España.

**SI SE EJECUTA ESTE NOTEBOOK EN LOCAL PUES SE DEBE CAMBIAR LA RUTA**

In [25]:
interest_rates = pd.read_excel('../input/datos-notebook-1/tipos_intereses.xlsx', "tipos_criterio_2")
interest_rates

,Año,Créditos c/p ≤ 250.000 euros,250.000 < Créditos c/p ≤ 1M euros,Créditos c/p > 1M euros,Créditos l/p ≤ 250.000 euros,250.000 < Créditos l/p ≤ 1M euros,Créditos l/p > 1M euros
0,2016,2.92000,1.885917,1.684083,3.003375,1.972000,1.681542
1,2017,2.43025,1.671833,1.633833,2.869708,1.893083,1.568542
2,2018,2.11250,1.551167,1.585583,2.855500,1.669875,1.488250
3,2019,1.98075,1.495833,1.469667,2.852500,1.619958,1.445625


Se eliminan las columnas de R* y zombi ya que se calcularán de nuevo con los tipos de intereses cargados en la celda anterior

In [26]:
sme.drop(columns=['r* 2019', 'r* 2018', 'r* 2017', 'zombi'])

,Código NIF,Nombre,ebit 2019,ebit 2018,ebit 2017,deuda_c/p 2019,deuda_c/p 2018,deuda_c/p 2017,deuda_c/p 2016,deuda_con_grupo_c/p 2019,...,drt_cp 2018,drt_cp 2017,drt_cp 2016,drt_lp 2019,drt_lp 2018,drt_lp 2017,drt_lp 2016,drt 2019,drt 2018,r 2019
0,A01045772,BAR RESTAURANTE LOS GUARANIS SA,-5106.77,-36794.05,-14717.25,8500,2123.97,1146.18,8938.08,n.d.,...,2123.97,1146.18,8938.08,8376.42,25335.05,33564.50,33670.97,16876.42,27459.02,0.082763
1,A01104157,EXPLOTACIONES SUBIJANA DE ALAVA SA,979628.32,1131808.94,841951.59,1259167.65,1802709.72,1615663.79,846415.71,n.d.,...,1802709.72,1615663.79,846415.71,13590231.27,8672240.51,9149877.46,10377394.89,14849398.92,10474950.23,0.015014
2,A01280577,BIASTERI TURISMO SA,209594.38,121434.18,178080.51,333764.6,743774.29,714753.39,557864.83,n.d.,...,743774.29,714753.39,557864.83,2785539.59,2030134.58,2253090.37,2553240.82,3119304.19,2773908.87,0.060682
3,A02003663,"HOTELES Y EDIFICIOS, SA",233388.3,172710.84,144877.19,n.d.,20972.88,100819.62,100027.24,n.d.,...,20972.88,100819.62,100027.24,826344.39,843899.78,870238.02,977568.56,826344.39,864872.66,0.023516
4,A02021574,HOTELES DE LA MANCHA SA,-11767.78,-89182.46,-119563.75,498100.37,464668.44,447806.23,416079.41,n.d.,...,464668.44,447806.23,416079.41,1461092.81,1666090.97,1698952.54,1680491.83,1959193.18,2130759.41,0.003723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9199,B99465692,HOSTELERA ZN30 SOCIEDAD LIMITADA.,8498.53,15012.04,13674.39,n.d.,16821.58,9326.49,2149.1,n.d.,...,16821.58,9326.49,2149.10,209309.61,274458.85,284028.83,286756.75,209309.61,291280.43,0.025195
9200,B99465932,TOP OF ROUSLAND SOCIEDAD LIMITADA.,-218173.64,-90294.6,-159608.07,n.d.,n.d.,97.1,n.d.,n.d.,...,0.00,97.10,0.00,0.00,408528.06,200240.82,116037.16,0.00,408528.06,0.001626
9201,B99470841,TARRACO GAVARRES SO FOOD SOCIEDAD LIMITADA.,247297.41,317916.51,489016.12,131051.56,127755.46,107869.82,n.d.,n.d.,...,127755.46,107869.82,0.00,778512.51,905247.20,1010515.85,1245038.17,909564.07,1033002.66,0.023870
9202,B99472052,PLAZA SITIOS 2017 SL.,143243.97,-50508.56,-198026.07,n.d.,19817.05,n.d.,n.d.,n.d.,...,19817.05,0.00,0.00,160757.33,122950.96,321994.16,228590.70,160757.33,142768.01,0.024934


Se vuelve a calcular R* (**ecuación 9**) para los años 2019, 2018 y 2017 y para cada una de las pymes del dataframe: **sme**

In [27]:
sme['r* 2019'] = sme[['drt_lp 2018', 'drt_cp 2018']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2019), axis=1)
sme['r* 2018'] = sme[['drt_lp 2017', 'drt_cp 2017']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2018), axis=1)
sme['r* 2017'] = sme[['drt_lp 2016', 'drt_cp 2016']].apply(lambda row: calculateMinInterestPayment(row[0], row[1], 2017), axis=1)

In [28]:
sme.head()

,Código NIF,Nombre,ebit 2019,ebit 2018,ebit 2017,deuda_c/p 2019,deuda_c/p 2018,deuda_c/p 2017,deuda_c/p 2016,deuda_con_grupo_c/p 2019,...,drt_lp 2018,drt_lp 2017,drt_lp 2016,drt 2019,drt 2018,r 2019,r* 2019,r* 2018,r* 2017,zombi
0,A01045772,BAR RESTAURANTE LOS GUARANIS SA,-5106.77,-36794.05,-14717.25,8500,2123.97,1146.18,8938.08,n.d.,...,25335.05,33564.50,33670.97,16876.42,27459.02,0.082763,768.311219,991.058293,1272.257431,0
1,A01104157,EXPLOTACIONES SUBIJANA DE ALAVA SA,979628.32,1131808.94,841951.59,1259167.65,1802709.72,1615663.79,846415.71,n.d.,...,8672240.51,9149877.46,10377394.89,14849398.92,10474950.23,0.015014,157648.084259,169916.893965,190462.913934,0
2,A01280577,BIASTERI TURISMO SA,209594.38,121434.18,178080.51,333764.6,743774.29,714753.39,557864.83,n.d.,...,2030134.58,2253090.37,2553240.82,3119304.19,2773908.87,0.060682,41750.656749,47290.146666,53454.674045,0
3,A02003663,"HOTELES Y EDIFICIOS, SA",233388.3,172710.84,144877.19,n.d.,20972.88,100819.62,100027.24,n.d.,...,843899.78,870238.02,977568.56,826344.39,864872.66,0.023516,14535.123541,18924.499732,22198.447411,0
4,A02021574,HOTELES DE LA MANCHA SA,-11767.78,-89182.46,-119563.75,498100.37,464668.44,447806.23,416079.41,n.d.,...,1666090.97,1698952.54,1680491.83,1959193.18,2130759.41,0.003723,32003.380813,34135.352309,36105.081266,1


Se calcula la etiqueta para cada una de las pymes disponibles en el dataframe: **sme**

In [29]:
sme['zombi'] = sme.apply(lambda row: detectZombieFirms([row['ebit 2019'], row['ebit 2018'], row['ebit 2017']],
                                                          [row['r* 2019'], row['r* 2018'], row['r* 2017']], row['r 2019'],
                                                          row['drt 2019'], row['drt 2018']), axis=1)

A continuación, se visualizan el número de pymes zombis identificadas por el segundo criterio propuesto en el estudio:

In [30]:
print("Pymes Totales:", sme.shape[0])
print("Pymes Zombi:", len(sme.loc[sme['zombi'] == 1]))
print("Pymes No Zombi:", len(sme.loc[sme['zombi'] == 0]))

Pymes Totales: 9204
Pymes Zombi: 564
Pymes No Zombi: 8640


Se guardan los códigos NIF de las empresas zombis identificadas con el segundo criterio para comparar el grado de semejanza de ambos criterios del estudio.

In [31]:
zombies_nif_2 = list(sme.loc[sme['zombi'] == 1, 'Código NIF'])
zombies_nif_2[0:5]

['A02021574', 'A03358744', 'A03462272', 'A04102547', 'A07051881']

## Comparación de las pymes zombis identificadas por criterio principal y alternativo

Para finalizar con el presente notebook, se calculan cuantas pymes identificadas por criterio principal del estudio son también identificadas como zombis por el criterio 2.

In [32]:
common_zombies = list(set(zombies_nif_1).intersection(zombies_nif_2))
print("Grado de semejanza entre el criterio alternativo y principal:",
      (len(common_zombies)/len(zombies_nif_2))*100, "%")

Grado de semejanza entre el criterio alternativo y principal: 100.0 %


Tal y como se puede observar en la anterior celda, todas aquellas 564 pymes que son detectadas como zombis por el segundo criterio lo son también por el criterio principal del estudio. Esto demuestra que ambos criterios sirven para identificar a las pymes zombis sin embargo, el segundo identifica una menor cantidad de pymes zombis en comparación al primero.